In [1]:
import re
from typing import Any, Callable, Dict

import httpx
import json
import time
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from datetime import datetime
from bs4 import BeautifulSoup
import constants as const

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from utils.road_tax import *

## Local Utils File
from utils.eda import *
from utils.utils import *
from utils.melissa_utils import *
from utils.Bhushan_utils import *
from utils.lta_omv_scraper import *

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_3551/1194282463.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
## Reading Dataset
train_df = pd.read_csv(r"./dataset/train.csv")

# clean model
train_df['model'] = train_df['model'].apply(lambda x:x.replace('(', ''))
train_df['reg_date_dt'] = train_df['reg_date'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))
train_df['reg_date_year'] = train_df['reg_date_dt'].apply(lambda x:x.year)
train_df['reg_date_month'] = train_df['reg_date_dt'].apply(lambda x:x.month)

# Adding additional feature for imputation later
train_df["model_make"] = train_df.model + "_" + train_df.make

## Scraping SG Carmart for Fuel Type

In [6]:
# Constants for the scraper
BASE_URL = "https://www.sgcarmart.com"
USED_CARS_URL = f"{BASE_URL}/used_cars/info.php"
# https://www.sgcarmart.com/used_cars/info.php?ID=1292132 -- style
NEW_CARS_URL = f"{BASE_URL}/new_cars/newcars_specs.php"
MISSING = "MISSING"

# Types
ListingId = int
CarCode, SubCode, FuelType = str, str, str
ScraperFunc = Callable[[Any], str]

# Global dictionaries
sub_code_dict: Dict[CarCode, SubCode] = {MISSING: MISSING}
fuel_type_dict: Dict[CarCode, FuelType] = {MISSING: MISSING}

In [21]:
train_df.columns

Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'indicative_price', 'price', 'reg_date_dt',
       'reg_date_year', 'reg_date_month', 'model_make'],
      dtype='object')

In [80]:
train_df[train_df.road_tax.isna()].opc_scheme.isna().sum()

2632

In [50]:
train_df[train_df.fuel_type.isna()]

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,lifespan,eco_category,features,accessories,indicative_price,price,reg_date_dt,reg_date_year,reg_date_month,model_make
0,1292132,Land Rover Range Rover Velar 3.0A Si6 R-Dynami...,land rover,range,"1 owner, no repairs needed! it looks great, in...",2018.0,NaN,08-mar-2018,suv,parf car,...,NaN,uncategorized,3l supercharged v6 p380 engine at 375bhp/450nm...,"2 x massage/memory/cooling & warmer seat, rear...",NaN,193788.0,2018-03-08,2018,3,range_land rover
1,1294696,Mercedes-Benz C-Class C200 Sport Premium Sunroof,mercedes-benz,c200,rare beautiful white c200 sport premium sunroo...,2017.0,NaN,28-dec-2017,luxury sedan,"parf car, premium ad car",...,NaN,uncategorized,"2.0l 4 cylinders inline turbocharged engine, p...","multi function steering, electric tailgate, re...",NaN,96800.0,2017-12-28,2017,12,c200_mercedes-benz
2,1311717,Honda Odyssey 2.4A (COE till 09/2027),honda,odyssey,comes with warranty. full service done.,2007.0,NaN,19-sep-2007,mpv,"coe car, premium ad car, low mileage car",...,NaN,uncategorized,"2.4l k24a 4 cylinders inline dohc i-vtec, 5 sp...","cruise control, touchscreen audio, reverse cam...",NaN,39800.0,2007-09-19,2007,9,odyssey_honda
3,1310068,Toyota Corolla Altis 1.6A (COE till 12/2028),toyota,altis,0,2008.0,NaN,15-dec-2008,mid-sized sedan,"coe car, premium ad car",...,NaN,uncategorized,super fuel efficient 1.6l 16 valves dohc vvt-i...,"leather seats, pioneer dvd audio system with r...",NaN,44800.0,2008-12-15,2008,12,altis_toyota
4,1325280,Lexus GS300 (COE till 06/2026),lexus,gs,wear and tear done up. well maintained and reg...,2006.0,NaN,22-dec-2006,luxury sedan,"coe car, premium ad car",...,NaN,uncategorized,"powerful 3.0l v6 engine, 227bhp, 6 speed , key...",premium upholstery electric seats. memory seat...,NaN,25800.0,2006-12-22,2006,12,gs_lexus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1329201,Mercedes-Benz C-Class C180 (COE till 06/2031),mercedes-benz,c180,elegant looking and very well maintained by ow...,2011.0,NaN,14-jul-2011,luxury sedan,"coe car, premium ad car, low mileage car",...,NaN,uncategorized,inline 4 16 valves rwd engine displacement 179...,new battery new throttle body new recording ca...,NaN,85000.0,2011-07-14,2011,7,c180_mercedes-benz
24996,1285898,Bentley Bentayga 4.0A V8,bentley,bentayga,"ceo car chauffeur driven, always sheltered par...",2018.0,23-aug-2018,29-apr-2020,suv,"parf car, direct owner sale, rare & exotic",...,NaN,uncategorized,"4.0 liter twin turbocharged v8 engine, produci...","panoramic roof, 8""touchscreen display, navigat...",NaN,412888.0,2020-04-29,2020,4,bentayga_bentley
24997,1306309,Mercedes-Benz A-Class A180,mercedes-benz,a180,great for short term drive or to renew coe. op...,2014.0,NaN,01-apr-2015,hatchback,"parf car, premium ad car, low mileage car, sgc...",...,NaN,uncategorized,1.6l inline 4 turbocharged engine with 7 speed...,"dual electric seats with 3 memory settings, di...",NaN,23888.0,2015-04-01,2015,4,a180_mercedes-benz
24998,1306902,Audi A3 Sportback 1.0A TFSI S-tronic,audi,a3,sporty audi a3 in town with upgraded coil over...,2017.0,NaN,30-jun-2017,hatchback,parf car,...,NaN,uncategorized,"fuel efficient 3 cylinder turbocharged engine,...","bbs sport rim, audi mmi/bluetooth/audio sound ...",NaN,53300.0,2017-06-30,2017,6,a3_audi


In [61]:
def try_except_wrapper(scraper_func: ScraperFunc) -> ScraperFunc:
    """
    Run the given scraper function, but if it fails, return `MISSING`.
    """

    def wrapper(*args, **kwargs):
        try:
            return scraper_func(*args, **kwargs)
        except Exception as e:
            print(f"{scraper_func.__name__} failed with error: {e}")
            return MISSING

    return wrapper

# ID = train_df.listing_id[1]
ID = train_df[train_df.fuel_type.isna()].listing_id[1]
r = httpx.get(f"{USED_CARS_URL}?ID={ID}")
display(f"CAR LISTING :: {ID}")

'CAR LISTING :: 1294696'

In [62]:
content = BeautifulSoup(r.content, "html.parser")
parent = content.find(class_="twoRow_info")
link = str(parent.parent.find("a"))
car_code = re.search(r"CarCode=(.+?)\"", link).group(1).replace("'", "")

In [63]:
display(parent.parent.find("a"))

<a href="/new_cars/newcars_overview.php?CarCode=11578" target="_blank">Mercedes-Benz C-Class Saloon (2014-2018)</a>

In [67]:
r = httpx.get(f"{NEW_CARS_URL}?CarCode={car_code}")
content = BeautifulSoup(r.content, "html.parser")
list_element = content.find(id="submodels_ul_link").find_all("a")
sub_code_dict[car_code] = list_element

In [68]:
display(list_element)

[<a href="newcars_specs.php?CarCode=11578&amp;Subcode=4161" style="color:#DE0807;">C180 Avantgarde (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4162">C180 Exclusive (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4164">C200 Avantgarde (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4163">C180 AMG Line (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4165">C200 Exclusive (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4166">C200 AMG Line (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4168">C250 AMG Line (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4167">C250 Avantgarde (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=5508">C43 AMG 4MATIC (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4749">C63 AMG (A)</a>,
 <a href="newcars_specs.php?CarCode=11578&amp;Subcode=4750">C63 S AMG (A)</a>]

In [77]:
query = f"{NEW_CARS_URL}?CarCode={car_code}"
r = httpx.get(query)
content = BeautifulSoup(r.content, "html.parser")
fuel_type = content.find("td", text="Fuel Type").nextSibling.nextSibling.text
fuel_type_dict[car_code] = fuel_type

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_638/974785582.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fuel_type = content.find("td", text="Fuel Type").nextSibling.nextSibling.text


In [78]:
fuel_type

'Petrol'

In [105]:
pd.to_datetime(train_df.reg_date)[0]

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_638/4246275337.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(train_df.reg_date)[0]


Timestamp('2018-03-08 00:00:00')

## Road Tax Calculator

In [129]:
train_df.opc_scheme.unique()

array([nan, 'revised opc scheme . learn more about opc schemes.',
       'old opc scheme . learn more about opc schemes.', '1100'],
      dtype=object)

In [134]:
train_df.category.unique()

array(['parf car', 'parf car, premium ad car',
       'coe car, premium ad car, low mileage car',
       'coe car, premium ad car', 'coe car, consignment car',
       'parf car, rare & exotic, premium ad car, low mileage car',
       'parf car, premium ad car, low mileage car', '-', 'coe car',
       'parf car, premium ad car, sgcarmart warranty cars',
       'opc car, parf car',
       'imported used vehicle, coe car, rare & exotic, vintage cars',
       'parf car, premium ad car, hybrid cars',
       'coe car, low mileage car', 'imported used vehicle, parf car',
       'parf car, consignment car, sgcarmart warranty cars',
       'parf car, sgcarmart warranty cars', 'parf car, low mileage car',
       'premium ad car',
       'coe car, premium ad car, sgcarmart warranty cars',
       'parf car, electric cars',
       'parf car, almost new car, low mileage car, hybrid cars',
       'parf car, hybrid cars', 'parf car, direct owner sale',
       'almost new car, electric cars',
       'c

In [135]:
## Assumption -- Everything is a car 
def calc_power(power):
    base_road_tax = 200
    if power <= 7.5:
        road_tax = base_road_tax
    elif power <= 30:
        road_tax = (200 + 2 * (power - 7.5))
    elif power <= 230:
        road_tax = (250 + 3.75 * (power - 30))
    else:
        road_tax = (1525 + 10 * (power - 230)) 
    return road_tax * 0.782

def calc_engine_cc(engine_capacity):
    base_road_tax = 200
    if engine_capacity <= 600:
        road_tax = base_road_tax
    elif engine_capacity <= 1000:
        road_tax = 200 + 0.125 * (engine_capacity - 600)
    elif engine_capacity <= 1600:
        road_tax = 250 + 0.375 * (engine_capacity - 1000)
    elif engine_capacity <= 3000:
        road_tax = 475 + 0.75 * (engine_capacity - 1600)
    else:
        road_tax = 1525 + 1 * (engine_capacity - 3000)   
    return road_tax * 0.782

def check_annual_rate(date):
    year_2023 = pd.Timestamp('2023-01-01')
    year_2022 = pd.Timestamp('2022-01-01')
    if date >=year_2023 :
        return 700
    elif date >= year_2022:
        return 400
    else:
        return 200

def calculate_road_tax(engine_capacity, power, age_of_car, reg_date, scheme="Normal", fuel_type="Petrol"):
    """Calculate Singapore road tax based on engine capacity, age of car, vehicle scheme, and fuel type."""

    if fuel_type == "electric":
        road_tax = calc_power(power)
        road_tax += check_annual_rate(reg_date)
        
    elif fuel_type == "petrol-electric":
        road_tax = max(calc_power(power), calc_engine_cc(engine_capacity))

    elif fuel_type == "diesel":
        # https://nea.gov.sg/our-services/pollution-control/air-pollution/air-pollution-regulations
        # Assumption Euro IV compliant
        road_tax = min(525, (engine_capacity*0.625-100))
    else:
        # Base Road Tax Calculation by Engine Capacity
        road_tax = calc_engine_cc(engine_capacity)
    
    # Adjustments for Vehicle Scheme
    if scheme == "OPC":
        # up to $500 on annual road tax, subject to a minimum road tax payment of $70 per year.
        road_tax = min(road_tax - 500, 70) 

    return road_tax

# REF :: https://onemotoring.lta.gov.sg/content/onemotoring/home/buying/upfront-vehicle-costs/tax-structure.html

# Example usage:
engine_capacity = 1500  # in cc
power = 50
age_of_car = 5  # in years
scheme = "Normal"  # or "OPC"
fuel_type = "Petrol"  # "Petrol", "Diesel", or "Electric"

road_tax = calculate_road_tax(engine_capacity, power, age_of_car, scheme, fuel_type)
print(f"Road Tax for a car with {engine_capacity}cc, {age_of_car} years old, under {scheme} scheme, using {fuel_type}: S${road_tax}")


Road Tax for a car with 1500cc, 5 years old, under Normal scheme, using Petrol: S$342.125


## Creating the fit transform

### Vehicle Type

In [16]:
def vehicle_type_train_fit(df:pd.DataFrame, column_name:str):
    """
    returns encoded vehicle type to the df and the scale
    """
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(df[[column_name]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))
    
    return pd.concat([df, encoded_df], axis=1) , encoder

def vehicle_type_test_transform(df:pd.DataFrame, column_name:str, encoder):
    encoded_data = encoder.fit(df[[column_name]])
    return pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))

# Gets back the encoder
train_encoded, vehicle_type_encoder = vehicle_type_fit_transform(train_df, "type_of_vehicle")

In [15]:
# Getting the categories
vehicle_type_encoder.get_feature_names_out()

array(['type_of_vehicle_bus/mini bus', 'type_of_vehicle_hatchback',
       'type_of_vehicle_luxury sedan', 'type_of_vehicle_mid-sized sedan',
       'type_of_vehicle_mpv', 'type_of_vehicle_others',
       'type_of_vehicle_sports car', 'type_of_vehicle_stationwagon',
       'type_of_vehicle_suv', 'type_of_vehicle_truck',
       'type_of_vehicle_van'], dtype=object)

### Getting model_make dictionary

In [35]:
def model_make_train_fit(df, column_a, column_b, impute_type:str="median"):
    """
    """
    # Calculate the mean of column B grouped by column A
    values = df.groupby(column_a)[column_b].agg([impute_type]).rename(columns={impute_type: column_b})[column_b]

    # Iterate over each row in the DataFrame
    df[column_b] = df[column_b].fillna(df[column_a].map(values))
    
    return df, values

feature = "curb_weight"
ref_col = "model_make"
new_df , curb_weight_dict = model_make_train_fit(train_df,ref_col,feature)


In [36]:
curb_weight_dict

model_make
107_peugeot         805.0
116d_bmw           1350.0
116i_bmw           1340.0
118i_bmw           1320.0
120i_bmw           1460.0
                   ...   
zafira_opel        1538.0
zk6117h_yutong    11220.0
zk6119h_yutong    11720.0
zoe_renault        1480.0
zs_mg              1532.0
Name: curb_weight, Length: 809, dtype: float64

In [34]:
def model_make_test_transform(df, ref_dict, column_b):

    df[column_b] = df[column_b].fillna(df[column_a].map(values))
    
    return df